In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json
from os.path import join, isdir
from pathlib import Path
import datetime
import pytz

import pandas as pd
import geopandas as gpd
import numpy as np

from covidcaremap.data import (processed_data_path, 
                               external_data_path,
                               published_data_path,
                               local_data_path)

In [3]:
ccm_csv_df = pd.read_csv(
    published_data_path('us_healthcare_capacity-facility-CovidCareMap.csv'), 
    dtype={'HCRIS-Provider Number': object, 
           'DH-OBJECTID': object,
           'CCM_ID': object
          }
)

In [4]:
ccm_csv_df.head()

,Name,Hospital Type,Address,Address_2,City,State,Zipcode,County,Latitude,Longitude,...,ICU Bed Occupancy Rate,Staffed All Beds - SOURCE,Staffed ICU Beds - SOURCE,Licensed All Beds - SOURCE,All Bed Occupancy Rate - SOURCE,ICU Bed Occupancy Rate - SOURCE,CCM_ID,DH-ID,HCRIS-ID,HIFLD-ID
0,IU HEALTH UNIVERSITY HOSPITAL,GENERAL ACUTE CARE,550 UNIVERSITY BLVD,NaN,INDIANAPOLIS,IN,46202,MARION,39.77528,-86.17656,...,NaN,None,None,NaN,None,None,100,NaN,NaN,100.0
1,HEALTHSOUTH NEW ENGLAND REHABILIATION AT BEVERLY,REHABILITATION,"800 CUMMINGS CENTER, 1ST FL, UNIT147",NaN,BEVERLY,MA,1915,ESSEX,42.56099,-70.88724,...,NaN,None,None,HIFLD-BEDS,None,None,10001915,NaN,NaN,10001915.0
2,ST. JOSEPH'S HOSPITAL,CRITICAL ACCESS,12886 TROXLER AVENUE,NaN,HIGHLAND,IL,62249,MADISON,38.75458,-89.66760,...,NaN,None,None,NaN,None,None,100062249,NaN,NaN,100062249.0
3,HOSP SAN CARLOS BORROMEO,GENERAL ACUTE CARE,550 CONCEPCION VERA AYALA,NaN,MOCA,PR,676,MOCA,18.39019,-67.10997,...,0.73,HCRIS-Total Staffed Beds,HCRIS-ICU Total Staffed Beds,DH-NUM_LICENS,HCRIS-Total Bed Occupancy Rate,HCRIS-ICU Occupancy Rate,1000676,4294.0,400111.0,1000676.0
4,RIVERVIEW MEDICAL CENTER,GENERAL ACUTE CARE,1 RIVERVIEW PLAZA,NaN,RED BANK,NJ,7701,MONMOUTH,40.35398,-74.06384,...,0.42,HCRIS-Total Staffed Beds,HCRIS-ICU Total Staffed Beds,DH-NUM_LICENS,HCRIS-Total Bed Occupancy Rate,HCRIS-ICU Occupancy Rate,10007701,2512.0,310034.0,10007701.0


In [5]:
now_utc = datetime.datetime.now()
now_est = now_utc.astimezone(pytz.timezone("US/Eastern"))

current_time = now_est.strftime("%Y-%m-%d-%H%M%S")
current_date = now_est.strftime("%Y-%m-%d")
print(current_date, current_time)

2020-07-23 2020-07-23-122935


In [6]:
!curl https://bi.ahca.myflorida.com/t/ABICC/views/Public/ICUBedsHospital.csv -o {processed_data_path('florida/raw/')}/ICUBedsHospital_{current_date}.csv 
!curl https://bi.ahca.myflorida.com/t/ABICC/views/Public/HospitalBedsHospital.csv -o {processed_data_path('florida/raw/')}/HospitalBedsHospital_{current_date}.csv 
!curl https://bi.ahca.myflorida.com/t/ABICC/views/Public/COVIDHospitalizationsCounty.csv -o {processed_data_path('florida/raw/')}/COVIDHospitalizationsCounty_{current_date}.csv 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  176k  100  176k    0     0   112k      0  0:00:01  0:00:01 --:--:--  112k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 86459  100 86459    0     0  55636      0  0:00:01  0:00:01 --:--:-- 55600
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   753  100   753    0     0    699      0  0:00:01  0:00:01 --:--:--   699


In [7]:
fl_icu_raw = pd.read_csv(processed_data_path(f'florida/raw/ICUBedsHospital_{current_date}.csv'), encoding='utf-8', sep=',')
fl_hosp_raw = pd.read_csv(processed_data_path(f'florida/raw/HospitalBedsHospital_{current_date}.csv'), encoding='utf-8', sep=',')

In [8]:
fl_icu_raw.head()

,County,FileNumber,Measure Names,ProviderName,Measure Values
0,ALACHUA,100113,Adult ICU Census,UF HEALTH SHANDS HOSPITAL,243
1,ALACHUA,100113,Available Adult ICU,UF HEALTH SHANDS HOSPITAL,17
2,ALACHUA,100113,Available Adult ICU%,UF HEALTH SHANDS HOSPITAL,0.065384615
3,ALACHUA,100113,Total AdultICU Capacity,UF HEALTH SHANDS HOSPITAL,260
4,ALACHUA,100113,Pediatric ICU Census,UF HEALTH SHANDS HOSPITAL,33


In [9]:
fl_icu_raw = pd.pivot_table(fl_icu_raw, 
                        values='Measure Values',
                        index=['County','FileNumber','ProviderName'], 
                        columns=['Measure Names'], aggfunc=np.sum).sort_values(['County','FileNumber']).reset_index()

fl_hosp_raw = pd.pivot_table(fl_hosp_raw, 
                        values='Measure Values',
                        index=['County','FileNumber','ProviderName'], 
                        columns=['Measure Names'], aggfunc=np.sum).sort_values(['County','FileNumber']).reset_index()

In [10]:
icu_all_idx = fl_icu_raw[fl_icu_raw['County']=='All'].index
fl_icu_raw = pd.concat([fl_icu_raw.iloc[icu_all_idx],fl_icu_raw.drop(icu_all_idx, axis=0)], axis=0).reset_index(drop=True)

hosp_all_idx = fl_hosp_raw[fl_hosp_raw['County']=='All'].index
fl_hosp_raw = pd.concat([fl_hosp_raw.iloc[hosp_all_idx],fl_hosp_raw.drop(hosp_all_idx, axis=0)], axis=0).reset_index(drop=True)

In [11]:
fl_icu_raw.head()

Measure Names,County,FileNumber,ProviderName,Adult ICU Census,Available Adult ICU,Available Adult ICU%,Available Pediatric ICU,Available Pediatric ICU%,Pediatric ICU Census,Total AdultICU Capacity,Total PediatricICU Capacity
0,All,All,All,"5,212",990,0.159625927,249,0.396496815,379,"6,202",628
1,ALACHUA,100113,UF HEALTH SHANDS HOSPITAL,243,17,0.065384615,14,0.297872340,33,260,47
2,ALACHUA,100204,NORTH FLORIDA REGIONAL MEDICAL CENTER,45,1,0.021739130,0,0,0,46,0
3,ALACHUA,110025,UF HEALTH REHAB HOSPITAL,0,0,0,0,0,0,0,0
4,ALACHUA,120011,UF HEALTH SHANDS PSYCHIATRIC HOSPITAL,0,0,0,0,0,0,0,0


In [12]:
fl_icu_raw.to_csv(processed_data_path(f'florida/ICU_BedsHospital1_crosstab_{current_date}.csv'), index=False)
fl_hosp_raw.to_csv(processed_data_path(f'florida/Hospital_BedsHospital1_crosstab_{current_date}.csv'), index=False)

In [13]:
update_date = sorted([f.stem for f in Path(processed_data_path('florida/')).iterdir() if 'csv' in f.suffix], reverse=True)[0].split('_')[-1]
update_date

'2020-07-23'

In [14]:
fl_hosp = pd.read_csv(processed_data_path(f'florida/Hospital_BedsHospital1_crosstab_{update_date}.csv'), encoding='utf-8', sep=',')
fl_icu = pd.read_csv(processed_data_path(f'florida/ICU_BedsHospital1_crosstab_{update_date}.csv'), encoding='utf-8', sep=',')

In [15]:
fl_icu.shape, fl_hosp.shape

((308, 11), (308, 7))

In [16]:
fl_icu.head()

,County,FileNumber,ProviderName,Adult ICU Census,Available Adult ICU,Available Adult ICU%,Available Pediatric ICU,Available Pediatric ICU%,Pediatric ICU Census,Total AdultICU Capacity,Total PediatricICU Capacity
0,All,All,All,"5,212",990,0.159626,249,0.396497,379,"6,202",628
1,ALACHUA,100113,UF HEALTH SHANDS HOSPITAL,243,17,0.065385,14,0.297872,33,260,47
2,ALACHUA,100204,NORTH FLORIDA REGIONAL MEDICAL CENTER,45,1,0.021739,0,0.000000,0,46,0
3,ALACHUA,110025,UF HEALTH REHAB HOSPITAL,0,0,0.000000,0,0.000000,0,0,0
4,ALACHUA,120011,UF HEALTH SHANDS PSYCHIATRIC HOSPITAL,0,0,0.000000,0,0.000000,0,0,0


In [17]:
fl_hosp.drop(0,inplace=True), fl_icu.drop(0,inplace=True)

(None, None)

In [18]:
fl_hosp['Total Staffed Bed Capacity'] = fl_hosp['Total Staffed Bed Capacity'].apply(lambda x: int(x.replace(',','')))
fl_icu['Total AdultICU Capacity'] = fl_icu['Total AdultICU Capacity'].apply(lambda x: int(x.replace(',','')))
fl_hosp['Bed Census'] = fl_hosp['Bed Census'].apply(lambda x: int(x.replace(',','')))
fl_icu['Adult ICU Census'] = fl_icu['Adult ICU Census'].apply(lambda x: int(x.replace(',','')))

In [19]:
fl_hosp.head()

,County,FileNumber,ProviderName,Available,Available Capacity,Bed Census,Total Staffed Bed Capacity
1,ALACHUA,100113,UF HEALTH SHANDS HOSPITAL,187,0.178776,859,1046
2,ALACHUA,100204,NORTH FLORIDA REGIONAL MEDICAL CENTER,26,0.055085,446,472
3,ALACHUA,110025,UF HEALTH REHAB HOSPITAL,6,0.120000,44,50
4,ALACHUA,120011,UF HEALTH SHANDS PSYCHIATRIC HOSPITAL,15,0.182927,67,82
5,ALACHUA,23960071,SELECT SPECIALTY HOSPITAL GAINESVILLE,5,0.104167,43,48


In [20]:
fl_icu.head()

,County,FileNumber,ProviderName,Adult ICU Census,Available Adult ICU,Available Adult ICU%,Available Pediatric ICU,Available Pediatric ICU%,Pediatric ICU Census,Total AdultICU Capacity,Total PediatricICU Capacity
1,ALACHUA,100113,UF HEALTH SHANDS HOSPITAL,243,17,0.065385,14,0.297872,33,260,47
2,ALACHUA,100204,NORTH FLORIDA REGIONAL MEDICAL CENTER,45,1,0.021739,0,0.000000,0,46,0
3,ALACHUA,110025,UF HEALTH REHAB HOSPITAL,0,0,0.000000,0,0.000000,0,0,0
4,ALACHUA,120011,UF HEALTH SHANDS PSYCHIATRIC HOSPITAL,0,0,0.000000,0,0.000000,0,0,0
5,ALACHUA,23960071,SELECT SPECIALTY HOSPITAL GAINESVILLE,0,0,0.000000,0,0.000000,0,0,0


In [21]:
facility_match_df = pd.read_csv(processed_data_path('fl_facilitymatching_2020-07-22.csv'))

In [22]:
facility_match_df.tail()

,ProviderName,Matched CCM_ID
302,HALIFAX HEALTH UF HEALTH MEDICAL CENTER OF DEL...,CCM-FL-0008
303,HEALTHMARK REGIONAL MEDICAL CENTER,15632435
304,ASCENSION SACRED HEART EMERALD COAST,33132550
305,NORTHWEST FLORIDA COMMUNITY HOSPITAL,28732428
306,ORLANDO HEALTH ST. CLOUD HOSPITAL,38334769


In [23]:
facility_matched_dict = {k:v for k,v in facility_match_df[['ProviderName','Matched CCM_ID']].values}

In [24]:
facility_matched_dict

{'UF HEALTH SHANDS HOSPITAL': '35032610',
 'NORTH FLORIDA REGIONAL MEDICAL CENTER': '27832605',
 'UF HEALTH REHAB HOSPITAL': '34132608',
 'UF HEALTH SHANDS PSYCHIATRIC HOSPITAL': '79632606',
 'SELECT SPECIALTY HOSPITAL GAINESVILLE': 'CCM-FL-0001',
 'ED FRASER MEMORIAL HOSPITAL': '8832063',
 'NORTHEAST FLORIDA STATE HOSPITAL': '28432063',
 'ASCENSION SACRED HEART BAY': '2632401',
 'GULF COAST REGIONAL MEDICAL CENTER': '14632405',
 'EMERALD COAST BEHAVIORAL HOSPITAL': '9132405',
 'ENCOMPASS HEALTH REHABILITATION HOSPITAL OF PANAMA CITY': '15932405',
 'SELECT SPECIALTY HOSPITAL - PANAMA CITY': '34532401',
 'NORTH FLORIDA REGIONAL MEDICAL CENTER STARKE CAMPUS': '35932091',
 'HOLMES REGIONAL MEDICAL CENTER': '17432901',
 'PARRISH MEDICAL CENTER': '30932796',
 'ROCKLEDGE REGIONAL MEDICAL CENTER': '44032955',
 'CAPE CANAVERAL HOSPITAL': '5432931',
 'CIRCLES OF CARE, INC': '6532901',
 'SEA PINES REHABILITATION HOSPITAL AFFILIATE OF ENCOMPASS HEALTH': '16532901',
 'DEVEREUX FLORIDA': '7532940',

In [25]:
manual_override_data = pd.read_csv(
    external_data_path('covidcaremap-ushcsc-facility-manual-override.csv'),
    dtype={'HCRIS-Provider Number': object, 
           'DH-OBJECTID': object,
           'CCM_ID': object}
)

In [26]:
manual_override_cols = list(manual_override_data.columns)
manual_override_cols

['Name',
 'Hospital Type',
 'Address',
 'Address_2',
 'City',
 'State',
 'Zipcode',
 'County',
 'Latitude',
 'Longitude',
 'Staffed All Beds',
 'Staffed ICU Beds',
 'Licensed All Beds',
 'All Bed Occupancy Rate',
 'ICU Bed Occupancy Rate',
 'Staffed All Beds - SOURCE',
 'Staffed ICU Beds - SOURCE',
 'Licensed All Beds - SOURCE',
 'All Bed Occupancy Rate - SOURCE',
 'ICU Bed Occupancy Rate - SOURCE',
 'CCM_ID',
 'HIFLD-ID',
 'Manual Override Reason',
 'Manual Override New Data Source',
 'DH-OBJECTID',
 'HCRIS-Provider Number']

In [27]:
ccm_csv_df.columns

Index(['Name', 'Hospital Type', 'Address', 'Address_2', 'City', 'State',
       'Zipcode', 'County', 'Latitude', 'Longitude', 'Staffed All Beds',
       'Staffed ICU Beds', 'Licensed All Beds', 'All Bed Occupancy Rate',
       'ICU Bed Occupancy Rate', 'Staffed All Beds - SOURCE',
       'Staffed ICU Beds - SOURCE', 'Licensed All Beds - SOURCE',
       'All Bed Occupancy Rate - SOURCE', 'ICU Bed Occupancy Rate - SOURCE',
       'CCM_ID', 'DH-ID', 'HCRIS-ID', 'HIFLD-ID'],
      dtype='object')

In [28]:
manual_override_cols

['Name',
 'Hospital Type',
 'Address',
 'Address_2',
 'City',
 'State',
 'Zipcode',
 'County',
 'Latitude',
 'Longitude',
 'Staffed All Beds',
 'Staffed ICU Beds',
 'Licensed All Beds',
 'All Bed Occupancy Rate',
 'ICU Bed Occupancy Rate',
 'Staffed All Beds - SOURCE',
 'Staffed ICU Beds - SOURCE',
 'Licensed All Beds - SOURCE',
 'All Bed Occupancy Rate - SOURCE',
 'ICU Bed Occupancy Rate - SOURCE',
 'CCM_ID',
 'HIFLD-ID',
 'Manual Override Reason',
 'Manual Override New Data Source',
 'DH-OBJECTID',
 'HCRIS-Provider Number']

In [29]:
col_replace = {
    'DH-OBJECTID': 'DH-ID',
    'HCRIS-Provider Number': 'HCRIS-ID',
    'HIFLD-ID': 'HIFLD-ID'
}

manual_override_cols = [col_replace.get(n, n) for n in manual_override_cols]

In [30]:
manual_override_cols

['Name',
 'Hospital Type',
 'Address',
 'Address_2',
 'City',
 'State',
 'Zipcode',
 'County',
 'Latitude',
 'Longitude',
 'Staffed All Beds',
 'Staffed ICU Beds',
 'Licensed All Beds',
 'All Bed Occupancy Rate',
 'ICU Bed Occupancy Rate',
 'Staffed All Beds - SOURCE',
 'Staffed ICU Beds - SOURCE',
 'Licensed All Beds - SOURCE',
 'All Bed Occupancy Rate - SOURCE',
 'ICU Bed Occupancy Rate - SOURCE',
 'CCM_ID',
 'HIFLD-ID',
 'Manual Override Reason',
 'Manual Override New Data Source',
 'DH-ID',
 'HCRIS-ID']

In [31]:
override_dict = {
    'Staffed All Beds - SOURCE': f"FL AHCA {update_date}",
    'Staffed ICU Beds - SOURCE': f"FL AHCA {update_date}",
    'All Bed Occupancy Rate - SOURCE': f"FL AHCA {update_date}",
    'ICU Bed Occupancy Rate - SOURCE': f"FL AHCA {update_date}",
    "Manual Override Reason": f"More recent data from {update_date}",
    "Manual Override New Data Source": "FL AHCA"
}
list(override_dict.keys())

['Staffed All Beds - SOURCE',
 'Staffed ICU Beds - SOURCE',
 'All Bed Occupancy Rate - SOURCE',
 'ICU Bed Occupancy Rate - SOURCE',
 'Manual Override Reason',
 'Manual Override New Data Source']

In [32]:
fl_hosp['ProviderName']

1                              UF HEALTH SHANDS HOSPITAL
2                  NORTH FLORIDA REGIONAL MEDICAL CENTER
3                               UF HEALTH REHAB HOSPITAL
4                  UF HEALTH SHANDS PSYCHIATRIC HOSPITAL
5                  SELECT SPECIALTY HOSPITAL GAINESVILLE
                             ...                        
303            SELECT SPECIALTY HOSPITAL - DAYTONA BEACH
304    HALIFAX HEALTH UF HEALTH MEDICAL CENTER OF DEL...
305                   HEALTHMARK REGIONAL MEDICAL CENTER
306                 ASCENSION SACRED HEART EMERALD COAST
307                 NORTHWEST FLORIDA COMMUNITY HOSPITAL
Name: ProviderName, Length: 307, dtype: object

In [33]:
manual_override_cols

['Name',
 'Hospital Type',
 'Address',
 'Address_2',
 'City',
 'State',
 'Zipcode',
 'County',
 'Latitude',
 'Longitude',
 'Staffed All Beds',
 'Staffed ICU Beds',
 'Licensed All Beds',
 'All Bed Occupancy Rate',
 'ICU Bed Occupancy Rate',
 'Staffed All Beds - SOURCE',
 'Staffed ICU Beds - SOURCE',
 'Licensed All Beds - SOURCE',
 'All Bed Occupancy Rate - SOURCE',
 'ICU Bed Occupancy Rate - SOURCE',
 'CCM_ID',
 'HIFLD-ID',
 'Manual Override Reason',
 'Manual Override New Data Source',
 'DH-ID',
 'HCRIS-ID']

In [34]:
facility_matched_dict

{'UF HEALTH SHANDS HOSPITAL': '35032610',
 'NORTH FLORIDA REGIONAL MEDICAL CENTER': '27832605',
 'UF HEALTH REHAB HOSPITAL': '34132608',
 'UF HEALTH SHANDS PSYCHIATRIC HOSPITAL': '79632606',
 'SELECT SPECIALTY HOSPITAL GAINESVILLE': 'CCM-FL-0001',
 'ED FRASER MEMORIAL HOSPITAL': '8832063',
 'NORTHEAST FLORIDA STATE HOSPITAL': '28432063',
 'ASCENSION SACRED HEART BAY': '2632401',
 'GULF COAST REGIONAL MEDICAL CENTER': '14632405',
 'EMERALD COAST BEHAVIORAL HOSPITAL': '9132405',
 'ENCOMPASS HEALTH REHABILITATION HOSPITAL OF PANAMA CITY': '15932405',
 'SELECT SPECIALTY HOSPITAL - PANAMA CITY': '34532401',
 'NORTH FLORIDA REGIONAL MEDICAL CENTER STARKE CAMPUS': '35932091',
 'HOLMES REGIONAL MEDICAL CENTER': '17432901',
 'PARRISH MEDICAL CENTER': '30932796',
 'ROCKLEDGE REGIONAL MEDICAL CENTER': '44032955',
 'CAPE CANAVERAL HOSPITAL': '5432931',
 'CIRCLES OF CARE, INC': '6532901',
 'SEA PINES REHABILITATION HOSPITAL AFFILIATE OF ENCOMPASS HEALTH': '16532901',
 'DEVEREUX FLORIDA': '7532940',

In [35]:
facility_matched_dict

{'UF HEALTH SHANDS HOSPITAL': '35032610',
 'NORTH FLORIDA REGIONAL MEDICAL CENTER': '27832605',
 'UF HEALTH REHAB HOSPITAL': '34132608',
 'UF HEALTH SHANDS PSYCHIATRIC HOSPITAL': '79632606',
 'SELECT SPECIALTY HOSPITAL GAINESVILLE': 'CCM-FL-0001',
 'ED FRASER MEMORIAL HOSPITAL': '8832063',
 'NORTHEAST FLORIDA STATE HOSPITAL': '28432063',
 'ASCENSION SACRED HEART BAY': '2632401',
 'GULF COAST REGIONAL MEDICAL CENTER': '14632405',
 'EMERALD COAST BEHAVIORAL HOSPITAL': '9132405',
 'ENCOMPASS HEALTH REHABILITATION HOSPITAL OF PANAMA CITY': '15932405',
 'SELECT SPECIALTY HOSPITAL - PANAMA CITY': '34532401',
 'NORTH FLORIDA REGIONAL MEDICAL CENTER STARKE CAMPUS': '35932091',
 'HOLMES REGIONAL MEDICAL CENTER': '17432901',
 'PARRISH MEDICAL CENTER': '30932796',
 'ROCKLEDGE REGIONAL MEDICAL CENTER': '44032955',
 'CAPE CANAVERAL HOSPITAL': '5432931',
 'CIRCLES OF CARE, INC': '6532901',
 'SEA PINES REHABILITATION HOSPITAL AFFILIATE OF ENCOMPASS HEALTH': '16532901',
 'DEVEREUX FLORIDA': '7532940',

In [36]:
fl_icu.dtypes

County                          object
FileNumber                      object
ProviderName                    object
Adult ICU Census                 int64
Available Adult ICU              int64
Available Adult ICU%           float64
Available Pediatric ICU          int64
Available Pediatric ICU%       float64
Pediatric ICU Census             int64
Total AdultICU Capacity          int64
Total PediatricICU Capacity      int64
dtype: object

In [37]:
manual_override_data.head()

,Name,Hospital Type,Address,Address_2,City,State,Zipcode,County,Latitude,Longitude,...,Staffed ICU Beds - SOURCE,Licensed All Beds - SOURCE,All Bed Occupancy Rate - SOURCE,ICU Bed Occupancy Rate - SOURCE,CCM_ID,HIFLD-ID,Manual Override Reason,Manual Override New Data Source,DH-OBJECTID,HCRIS-Provider Number
0,FLORIDA HOSPITAL OCEANSIDE,REHABILITATION,264 S ATLANTIC AVE,NaN,ORMOND BEACH,FL,32176,VOLUSIA,29.28464,-81.03687,...,None,HIFLD-BEDS,None,None,13032176,13032176.0,More recent source,FL AHCA,NaN,NaN
1,FORT LAUDERDALE HOSPITAL,PSYCHIATRIC,1601 E LAS OLAS BLVD,NaN,FORT LAUDERDALE,FL,33301,BROWARD,26.11962,-80.12677,...,HCRIS-ICU Total Staffed Beds,HCRIS-Total Staffed Beds,HCRIS-Total Bed Occupancy Rate,HCRIS-ICU Occupancy Rate,13833301,13833301.0,More recent source,FL AHCA,NaN,104026.0
2,NAVAL HOSPITAL PENSACOLA,MILITARY,6000 WEST HIGHWAY 98,NaN,PENSACOLA,FL,32512,ESCAMBIA,30.39857,-87.29862,...,None,NaN,None,None,15,15.0,More recent source,FL AHCA,NaN,NaN
3,OLANDO VA MEDICAL CENTER,MILITARY,13800 VETERANS WAY,NaN,ORLANDO,FL,32827,ORANGE,28.36614,-81.27571,...,None,NaN,None,None,181032827,181032827.0,More recent source,FL AHCA,5583.0,NaN
4,JAMES A HALEY VETERANS' HOSPITAL,MILITARY,13000 BRUCE B DOWNS BLVD,NaN,TAMPA,FL,33612,HILLSBOROUGH,28.06441,-82.42928,...,None,HIFLD-BEDS,None,None,18933612,18933612.0,More recent source,FL AHCA,756.0,NaN


In [38]:
ccm_fl = ccm_csv_df[ccm_csv_df['State'] == 'FL']

In [39]:
ccm_fl.head()

,Name,Hospital Type,Address,Address_2,City,State,Zipcode,County,Latitude,Longitude,...,ICU Bed Occupancy Rate,Staffed All Beds - SOURCE,Staffed ICU Beds - SOURCE,Licensed All Beds - SOURCE,All Bed Occupancy Rate - SOURCE,ICU Bed Occupancy Rate - SOURCE,CCM_ID,DH-ID,HCRIS-ID,HIFLD-ID
90,ADVENTHEALTH ORLANDO,GENERAL ACUTE CARE,601 E ROLLINS ST,NaN,ORLANDO,FL,32803,ORANGE,28.57441,-81.37000,...,0.865000,FL AHCA 2020-07-23,FL AHCA 2020-07-23,HCRIS-Total Staffed Beds,FL AHCA 2020-07-23,FL AHCA 2020-07-23,10232803,804.0,100007.0,10232803.0
194,ADVENTHEALTH ALTAMONTE SPRINGS,GENERAL ACUTE CARE,601 E ALTAMONTE DR,NaN,ALTAMONTE SPRINGS,FL,32701,SEMINOLE,28.66538,-81.37016,...,0.857143,FL AHCA 2020-07-23,FL AHCA 2020-07-23,HIFLD-BEDS,FL AHCA 2020-07-23,FL AHCA 2020-07-23,10532701,NaN,NaN,10532701.0
310,ADVENTHEALTH APOPKA,GENERAL ACUTE CARE,201 N PARK AVE,NaN,APOPKA,FL,32703,ORANGE,28.68089,-81.50808,...,0.684211,FL AHCA 2020-07-23,FL AHCA 2020-07-23,HIFLD-BEDS,FL AHCA 2020-07-23,FL AHCA 2020-07-23,10832703,NaN,NaN,10832703.0
352,ADVENTHEALTH CONNERTON,LONG TERM CARE,9441 HEALTH CENTER DR,NaN,LAND O LAKES,FL,34637,PASCO,28.29527,-82.48991,...,NaN,FL AHCA 2020-07-23,FL AHCA 2020-07-23,DH-NUM_LICENS,FL AHCA 2020-07-23,FL AHCA 2020-07-23,10934637,5829.0,102026.0,10934637.0
424,ADVENTHEALTH CARROLLWOOD,GENERAL ACUTE CARE,7171 N DALE MABRY HWY,NaN,TAMPA,FL,33614,HILLSBOROUGH,28.01324,-82.50401,...,0.857143,FL AHCA 2020-07-23,FL AHCA 2020-07-23,DH-NUM_LICENS,FL AHCA 2020-07-23,FL AHCA 2020-07-23,11133614,752.0,100069.0,11133614.0


In [40]:
ccm_fl['Manual Override Reason'] = 'More recent source'
ccm_fl['Manual Override New Data Source'] = 'FL AHCA'
ccm_fl.loc[:,'Staffed All Beds'] = np.nan
ccm_fl.loc[:,'Staffed ICU Beds'] = np.nan
ccm_fl.loc[:,'All Bed Occupancy Rate'] = np.nan
ccm_fl.loc[:,'ICU Bed Occupancy Rate'] = np.nan

<ipython-input-40-919be17a7fa3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ccm_fl['Manual Override Reason'] = 'More recent source'
<ipython-input-40-919be17a7fa3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ccm_fl['Manual Override New Data Source'] = 'FL AHCA'
/usr/local/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

In [41]:
ccm_fl.T

,90,194,310,352,424,481,517,551,668,705,...,7051,7087,7146,7147,7148,7149,7150,7151,7152,7153
Name,ADVENTHEALTH ORLANDO,ADVENTHEALTH ALTAMONTE SPRINGS,ADVENTHEALTH APOPKA,ADVENTHEALTH CONNERTON,ADVENTHEALTH CARROLLWOOD,BAPTIST HOSPITAL,ADVENTHEALTH CELEBRATION,ADVENTHEALTH DELAND,ADVENTHEALTH EAST ORLANDO,ADVENTHEALTH FISH MEMORIAL,...,FISHERMEN'S COMMUNITY HOSPITAL,FLAGLER HOSPITAL,CLEVELAND CLINIC MARTIN SOUTH HOSPITAL,SELECT SPECIALTY HOSPITAL GAINESVILLE,PALM POINT BEHAVIORAL HEALTH,FORT LAUDERDALE BEHAVIORAL HEALTH CENTER,ASCENSION SACRED HEART PENSACOLA,UF HEALTH LEESBURG SENIOR BEHAVIORAL HEALTH CE...,LAKE CITY MEDICAL CENTER SUWANNEE CAMPUS,HALIFAX HEALTH UF HEALTH MEDICAL CENTER OF DEL...
Hospital Type,GENERAL ACUTE CARE,GENERAL ACUTE CARE,GENERAL ACUTE CARE,LONG TERM CARE,GENERAL ACUTE CARE,GENERAL ACUTE CARE,GENERAL ACUTE CARE,GENERAL ACUTE CARE,GENERAL ACUTE CARE,GENERAL ACUTE CARE,...,GENERAL ACUTE CARE,GENERAL ACUTE CARE,GENERAL ACUTE CARE,LONG TERM CARE,PSYCHIATRIC,PSYCHIATRIC,GENERAL ACUTE CARE,PSYCHIATRIC,OTHER,GENERAL ACUTE CARE
Address,601 E ROLLINS ST,601 E ALTAMONTE DR,201 N PARK AVE,9441 HEALTH CENTER DR,7171 N DALE MABRY HWY,1000 W MORENO ST,400 CELEBRATION PLACE,701 W PLYMOUTH AVE,7727 LAKE UNDERHILL RD,1055 SAXON BLVD,...,3301 OVERSEAS HWY,400 HEALTH PARK BLVD,2100 SE Salerno Rd #6503,1600 SW Archer Rd 5th Floor,2355 Truman Scarborough Way,5757 N Dixie Hwy,5151 N 9th Ave,"700 N Palmetto St, 3rd Floor",1100 11th St SW,3300 Halifax Crossing Blvd
Address_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City,ORLANDO,ALTAMONTE SPRINGS,APOPKA,LAND O LAKES,TAMPA,PENSACOLA,CELEBRATION,DELAND,ORLANDO,ORANGE CITY,...,MARATHON,SAINT AUGUSTINE,STUART,GAINESVILLE,TITUSVILLE,OAKLAND PARK,PENSACOLA,LEESBURG,LIVE OAK,DELTONA
State,FL,FL,FL,FL,FL,FL,FL,FL,FL,FL,...,FL,FL,FL,FL,FL,FL,FL,FL,FL,FL
Zipcode,32803,32701,32703,34637,33614,32501,34747,32721,32822,32763,...,33050,32086,34997,32608,32796,33334,32504,34748,32064,32725
County,ORANGE,SEMINOLE,ORANGE,PASCO,HILLSBOROUGH,ESCAMBIA,OSCEOLA,VOLUSIA,ORANGE,VOLUSIA,...,MONROE,ST. JOHNS,MARTIN,ALACHUA,BREVARD,BROWARD,ESCAMBIA,LAKE,SUWANNEE,VOLUSIA
Latitude,28.5744,28.6654,28.6809,28.2953,28.0132,30.4294,28.3281,29.0436,28.54,28.9144,...,24.7104,29.863,27.1339,29.6398,28.6405,26.1979,30.476,28.8183,30.2917,28.9574
Longitude,-81.37,-81.3702,-81.5081,-82.4899,-82.504,-87.2304,-81.5409,-81.3173,-81.2815,-81.2852,...,-81.0944,-81.3171,-80.2248,-82.3435,-80.8302,-80.1333,-87.2124,-81.8745,-83.0028,-81.2556


In [42]:
l = []
not_added = []
for p in fl_hosp['ProviderName']:
    
    try: ccm_id = str(facility_matched_dict[p])
    except Exception as exc: 
        print('not found: ',exc)
        ccm_id = ''
    found_ccm_record = ccm_csv_df[ccm_csv_df['CCM_ID'] == ccm_id]
    #if ccm_id == 'FL-CCM-0001': print(p, found_ccm_record)
    
    # ignore adding new facilities for now, only update existing matched facilities
    if len(ccm_csv_df[ccm_csv_df['CCM_ID'] == ccm_id]) > 0:
        
        total_staffed_beds = fl_hosp[fl_hosp['ProviderName'] == p]['Total Staffed Bed Capacity'].values[0]
        icu_staffed_beds = fl_icu[fl_icu['ProviderName'] == p]['Total AdultICU Capacity'].values[0]
        
        if total_staffed_beds > 0:
            hosp_occupancy = fl_hosp[fl_hosp['ProviderName'] == p]['Bed Census'].values[0]/total_staffed_beds
#         elif total_staffed_beds == 0: hosp_occupancy = 0
        else: hosp_occupancy = np.nan
            
        if icu_staffed_beds > 0:
            icu_occupancy = fl_icu[fl_icu['ProviderName'] == p]['Adult ICU Census'].values[0]/icu_staffed_beds
#         elif icu_staffed_beds == 0: icu_occupancy = 0
        else: icu_occupancy = np.nan
        
        override_dict['Staffed All Beds'] = fl_hosp[fl_hosp['ProviderName'] == p]['Total Staffed Bed Capacity'].values[0]
        override_dict['Staffed ICU Beds'] = fl_icu[fl_icu['ProviderName'] == p]['Total AdultICU Capacity'].values[0]
        override_dict['All Bed Occupancy Rate'] = hosp_occupancy
        override_dict['ICU Bed Occupancy Rate'] = icu_occupancy
        override_dict['Name'] = p
    
        row_dict = {}

        for col in list(manual_override_cols):
#             print(col)
            # update data for col fields from FL
            if col in list(override_dict.keys()):
                row_dict[col] = override_dict[col]
            # add the other col fields unchanged
            elif len(found_ccm_record[col].values)>0:
                row_dict[col] = found_ccm_record[col].values[0]
            else:
                row_dict[col] = np.nan
        l.append(row_dict)
    else: not_added.append((p,ccm_id))

In [43]:
 not_added

[]

In [44]:
new_overrides_df = pd.DataFrame(l)

In [45]:
new_overrides_df.head()

,Name,Hospital Type,Address,Address_2,City,State,Zipcode,County,Latitude,Longitude,...,Staffed ICU Beds - SOURCE,Licensed All Beds - SOURCE,All Bed Occupancy Rate - SOURCE,ICU Bed Occupancy Rate - SOURCE,CCM_ID,HIFLD-ID,Manual Override Reason,Manual Override New Data Source,DH-ID,HCRIS-ID
0,UF HEALTH SHANDS HOSPITAL,GENERAL ACUTE CARE,1600 SW ARCHER RD,NaN,GAINESVILLE,FL,32610,ALACHUA,29.63999,-82.34286,...,FL AHCA 2020-07-23,DH-NUM_LICENS,FL AHCA 2020-07-23,FL AHCA 2020-07-23,35032610,35032610.0,More recent data from 2020-07-23,FL AHCA,695.0,100113.0
1,NORTH FLORIDA REGIONAL MEDICAL CENTER,GENERAL ACUTE CARE,6500 NEWBERRY RD,NaN,GAINESVILLE,FL,32605,ALACHUA,29.66061,-82.41217,...,FL AHCA 2020-07-23,DH-NUM_LICENS,FL AHCA 2020-07-23,FL AHCA 2020-07-23,27832605,27832605.0,More recent data from 2020-07-23,FL AHCA,696.0,100204.0
2,UF HEALTH REHAB HOSPITAL,LONG TERM CARE,2708 SW ARCHER RD,NaN,GAINESVILLE,FL,32608,ALACHUA,29.63213,-82.36395,...,FL AHCA 2020-07-23,DH-NUM_LICENS,FL AHCA 2020-07-23,FL AHCA 2020-07-23,34132608,34132608.0,More recent data from 2020-07-23,FL AHCA,6521.0,NaN
3,UF HEALTH SHANDS PSYCHIATRIC HOSPITAL,PSYCHIATRIC,4101 NW 89TH BLVD,NaN,GAINESVILLE,FL,32606,ALACHUA,29.69182,-82.43620,...,FL AHCA 2020-07-23,HIFLD-BEDS,FL AHCA 2020-07-23,FL AHCA 2020-07-23,79632606,79632606.0,More recent data from 2020-07-23,FL AHCA,NaN,NaN
4,SELECT SPECIALTY HOSPITAL GAINESVILLE,LONG TERM CARE,1600 SW Archer Rd 5th Floor,NaN,GAINESVILLE,FL,32608,ALACHUA,29.63982,-82.34346,...,FL AHCA 2020-07-23,Manual Override,FL AHCA 2020-07-23,FL AHCA 2020-07-23,CCM-FL-0001,NaN,More recent data from 2020-07-23,FL AHCA,NaN,NaN


In [46]:
manual_override_data = pd.concat([ccm_fl, manual_override_data, new_overrides_df]).drop_duplicates('CCM_ID' , keep='last')

In [47]:
manual_override_data.iloc[44:80]

,Name,Hospital Type,Address,Address_2,City,State,Zipcode,County,Latitude,Longitude,...,All Bed Occupancy Rate - SOURCE,ICU Bed Occupancy Rate - SOURCE,CCM_ID,DH-ID,HCRIS-ID,HIFLD-ID,Manual Override Reason,Manual Override New Data Source,DH-OBJECTID,HCRIS-Provider Number
44,ARKANSAS VALLEY REGIONAL MEDICAL CENTER,GENERAL ACUTE CARE,1100 CARSON AVENUE,NaN,LA JUNTA,CO,81050,OTERO,37.97807,-103.54958,...,Manual Override,Manual Override,8381050,NaN,NaN,8381050.0,ICU and total beds way off,https://www.avrmc.org/getpage.php?name=about,636,61336
45,LAGUNA HONDA HOSPITAL AND REHABILITATION CENTER,GENERAL ACUTE CARE,375 LAGUNA HONDA BOULEVARD,NaN,SAN FRANCISCO,CA,94116,SAN FRANCISCO,37.74931,-122.45689,...,None,None,25094116,NaN,NaN,25094116.0,Geolocation is off,OSM,NaN,NaN
46,ST. FRANCIS MEMORIAL HOSPITAL,GENERAL ACUTE CARE,900 HYDE STREET,NaN,SAN FRANCISCO,CA,94109,SAN FRANCISCO,37.78963,-122.41653,...,HCRIS-Total Bed Occupancy Rate,HCRIS-ICU Occupancy Rate,45094109,NaN,NaN,45094109.0,Geolocation is off,OSM,503,50152
0,UF HEALTH SHANDS HOSPITAL,GENERAL ACUTE CARE,1600 SW ARCHER RD,NaN,GAINESVILLE,FL,32610,ALACHUA,29.63999,-82.34286,...,FL AHCA 2020-07-23,FL AHCA 2020-07-23,35032610,695.0,100113.0,35032610.0,More recent data from 2020-07-23,FL AHCA,NaN,NaN
1,NORTH FLORIDA REGIONAL MEDICAL CENTER,GENERAL ACUTE CARE,6500 NEWBERRY RD,NaN,GAINESVILLE,FL,32605,ALACHUA,29.66061,-82.41217,...,FL AHCA 2020-07-23,FL AHCA 2020-07-23,27832605,696.0,100204.0,27832605.0,More recent data from 2020-07-23,FL AHCA,NaN,NaN
2,UF HEALTH REHAB HOSPITAL,LONG TERM CARE,2708 SW ARCHER RD,NaN,GAINESVILLE,FL,32608,ALACHUA,29.63213,-82.36395,...,FL AHCA 2020-07-23,FL AHCA 2020-07-23,34132608,6521.0,NaN,34132608.0,More recent data from 2020-07-23,FL AHCA,NaN,NaN
3,UF HEALTH SHANDS PSYCHIATRIC HOSPITAL,PSYCHIATRIC,4101 NW 89TH BLVD,NaN,GAINESVILLE,FL,32606,ALACHUA,29.69182,-82.43620,...,FL AHCA 2020-07-23,FL AHCA 2020-07-23,79632606,NaN,NaN,79632606.0,More recent data from 2020-07-23,FL AHCA,NaN,NaN
4,SELECT SPECIALTY HOSPITAL GAINESVILLE,LONG TERM CARE,1600 SW Archer Rd 5th Floor,NaN,GAINESVILLE,FL,32608,ALACHUA,29.63982,-82.34346,...,FL AHCA 2020-07-23,FL AHCA 2020-07-23,CCM-FL-0001,NaN,NaN,NaN,More recent data from 2020-07-23,FL AHCA,NaN,NaN
5,ED FRASER MEMORIAL HOSPITAL,GENERAL ACUTE CARE,159 N 3RD ST,NaN,MACCLENNY,FL,32063,BAKER,30.28600,-82.11926,...,FL AHCA 2020-07-23,FL AHCA 2020-07-23,8832063,4328.0,100134.0,8832063.0,More recent data from 2020-07-23,FL AHCA,NaN,NaN
6,NORTHEAST FLORIDA STATE HOSPITAL,PSYCHIATRIC,7487 S STATE RD 121,NaN,MACCLENNY,FL,32063,BAKER,30.23741,-82.13398,...,FL AHCA 2020-07-23,FL AHCA 2020-07-23,28432063,4615.0,104007.0,28432063.0,More recent data from 2020-07-23,FL AHCA,NaN,NaN


In [48]:
manual_override_data['DH-OBJECTID'] = manual_override_data['DH-OBJECTID'].combine_first(manual_override_data['DH-ID'])
manual_override_data['HCRIS-Provider Number'] = manual_override_data['HCRIS-Provider Number'].combine_first(manual_override_data['HCRIS-ID'])

In [49]:
manual_override_data.drop(['DH-ID', 'HCRIS-ID'], inplace=True, axis=1)

In [50]:
# Ensure coordinates are rounded
manual_override_data['Latitude'] = manual_override_data['Latitude'].round(5)
manual_override_data['Longitude'] = manual_override_data['Longitude'].round(5)

In [51]:
manual_override_data.to_csv(external_data_path('covidcaremap-ushcsc-facility-manual-override.csv'), index=False)